In [305]:
import pandas as pd
import os

os.chdir("/home/matt/projects/kaggle-fast-or-slow")

SUBMISSION_IDS = ["z7mrv7xe_epoch2_xla.csv", "z7mrv7xe_epoch3_xla.csv","r3y1ck2b_xla.csv"]

dataframes = [pd.read_csv(f"submissions/{s}", index_col=0) for s in SUBMISSION_IDS]

# Numeric normalization and averaging
df = dataframes[0].T
df = (df - df.min(axis=0)) / (df.max(axis=0) - df.min(axis=0))

for d in dataframes[1:]:
    dt = d.T
    df = df + (dt - dt.min(axis=0)) / (dt.max(axis=0) - dt.min(axis=0))


df

,layout:xla:random:05ae41e26dd3c4c06390371a0423233c,layout:xla:random:3e7156ac468dfb75cf5c9615e1e5887d,layout:xla:random:5335ed13823b0a518ee3c79ba4425f34,layout:xla:random:937ee0eb0d5d6151b7b8252933b5c1c9,layout:xla:random:cd708819d3f5103afd6460b15e74eaf3,layout:xla:random:db59a991b7c607634f13570d52ce885f,layout:xla:random:e8a3a1401b5e79f66d7037e424f3b6df,layout:xla:random:fbaa8bb6a1aed9988281085c91065c05,layout:xla:default:05ae41e26dd3c4c06390371a0423233c,layout:xla:default:3e7156ac468dfb75cf5c9615e1e5887d,layout:xla:default:5335ed13823b0a518ee3c79ba4425f34,layout:xla:default:937ee0eb0d5d6151b7b8252933b5c1c9,layout:xla:default:cd708819d3f5103afd6460b15e74eaf3,layout:xla:default:db59a991b7c607634f13570d52ce885f,layout:xla:default:e8a3a1401b5e79f66d7037e424f3b6df,layout:xla:default:fbaa8bb6a1aed9988281085c91065c05
0,1.187281,1.105539,1.050641,2.941167,0.879672,1.625199,2.046487,0.941573,1.299070,0.348943,0.998165,2.850854,0.759415,0.421675,0.925948,1.768461
1,0.331188,2.401481,1.863003,0.627271,1.121614,2.358752,1.907146,0.239923,0.090812,0.365607,1.066895,0.033220,0.393775,2.192003,2.085601,2.275468
2,1.079389,2.930204,0.710022,2.783000,2.557601,2.380062,1.488579,1.672837,0.477931,2.710189,0.346816,2.854082,2.963199,2.277608,1.122368,2.506815
3,1.193347,1.352320,0.757219,0.745113,0.902125,2.584659,1.497767,1.965625,0.911542,0.702103,0.765830,0.032507,2.172429,1.748131,1.344656,2.258227
4,1.062618,1.210935,1.135140,1.633529,1.168742,2.314322,1.774789,2.559936,0.538710,1.002461,0.765406,0.033938,2.622904,1.833872,0.919848,2.937535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.269691,2.225880,2.424331,1.975667,1.208616,1.929040,1.378912,1.558204,0.691641,1.213830,0.511630,2.731008,1.826747,0.591581,1.932105,2.466889
997,0.528263,2.118984,2.069938,2.834625,1.118349,0.460308,1.321754,2.028823,0.580440,2.526655,1.308472,0.022667,0.729323,0.203612,1.263269,2.487984
998,0.440013,2.975829,1.129834,0.888631,2.532618,0.892684,1.564022,1.646249,0.697995,2.764838,0.731203,0.031047,0.477772,0.203566,1.258714,1.952533
999,0.679177,1.325806,0.358585,0.841297,1.101759,1.971436,1.751695,0.736610,0.485200,0.335931,0.173578,0.035928,0.276056,1.269142,0.681634,2.214902


In [310]:
dataframes[0].index

Index(['layout:xla:random:05ae41e26dd3c4c06390371a0423233c',
       'layout:xla:random:3e7156ac468dfb75cf5c9615e1e5887d',
       'layout:xla:random:5335ed13823b0a518ee3c79ba4425f34',
       'layout:xla:random:937ee0eb0d5d6151b7b8252933b5c1c9',
       'layout:xla:random:cd708819d3f5103afd6460b15e74eaf3',
       'layout:xla:random:db59a991b7c607634f13570d52ce885f',
       'layout:xla:random:e8a3a1401b5e79f66d7037e424f3b6df',
       'layout:xla:random:fbaa8bb6a1aed9988281085c91065c05',
       'layout:xla:default:05ae41e26dd3c4c06390371a0423233c',
       'layout:xla:default:3e7156ac468dfb75cf5c9615e1e5887d',
       'layout:xla:default:5335ed13823b0a518ee3c79ba4425f34',
       'layout:xla:default:937ee0eb0d5d6151b7b8252933b5c1c9',
       'layout:xla:default:cd708819d3f5103afd6460b15e74eaf3',
       'layout:xla:default:db59a991b7c607634f13570d52ce885f',
       'layout:xla:default:e8a3a1401b5e79f66d7037e424f3b6df',
       'layout:xla:default:fbaa8bb6a1aed9988281085c91065c05'],
      dtype='ob

In [312]:
import numpy as np
from numpy.typing import NDArray
from typing import Tuple, Union, List, Any



def borda_count_np(X, min_score: float = 1, k: int = None):
    """Given N ranked ids lists of length P compute the number of
    extractions on top-k positions and the mean position for each id.
    Sort the element ids (in reversed DECREASING order) based on the
    Borda Extraction Scores (BES). IDs having the same BES will be sorted
    by their increasing average position (i.e. the average of the position
    of each ID in all the rankings).
    Parameters
    ----------
    X : Numpy array of integer with shape [N, P] or List of Lists
        The list of rankings
    min_score: float, optional
        Minimum score considered for IDs (Default is 1)
    k : int, optional
        Top-K positions to select.
        Default is None, which means that all the positions will be
        selected (i.e. k = P)
    Returns
    -------
    Borda
        A tuple containing three NDarray:
        1. Sorted IDs, i.e. the Borda Ranking;
        2. The number of Extractions, i.e. Count;
        3. The average positions of each ID.
    Raises
    ------
    ValueError
        If k < 1 or k > p
    ValueError
        If X contains duplicate IDs
    """

    x_np = np.asarray(X)  # make sure X is a Numpy array for efficiency
    n, p = x_np.shape
    if k is None:
        k = p
        
    if not 1 <= k <= p:
        raise ValueError(f"k must be in [1, {p}], {k} given instead!")
    # Verify that ranked lists do no contain repetitions
    unique_ids = np.asarray([len(set(r)) for r in x_np[:]])
    if len(unique_ids[unique_ids < p]):
        raise ValueError(
            f"IDs in list of rankings are not unique! Unique IDs found: {unique_ids}"
        )

    positions = np.argsort(x_np, axis=1)
    # Select TopK by forcing positions of others to last position
    positions[positions >= k] = p
    # Calculate scores of extractions in each ranking based on their relative position
    exts = (p - positions - 1 + min_score).sum(axis=0)
    # Count total number of valid extractions
    counts = (positions < k).sum(axis=0)
    # Reset all non-valid positions (>=k) for TopK selection
    positions[positions >= k] = 0
    # Calculate Average positions of each element in each ranking
    non_zero_counts = counts != 0
    avg_positions = positions.sum(axis=0) / np.where(non_zero_counts, counts, 1)
    avg_positions[~non_zero_counts] = p - 1
    # Sort rankings based on average positions, and extraction scores
    idx = np.lexsort((avg_positions, exts))[::-1]

    return idx, counts[idx], avg_positions[idx]


bordas = {}
for i in range(dataframes[0].shape[0]):
    index = dataframes[0].index[i]
    lists = [d.loc[index, :] for d in dataframes]
    for i in range(len(lists)):
        lists[i] = lists[i][~np.isnan(lists[i])].argsort()
    
    lists = np.stack(lists)
        
    borda, _, _ = borda_count_np(lists)
    bordas[index] = borda
    
    
borda_results = {}
for k, v in bordas.items():
    borda_results[k] = ";".join([str(x) for x in v.tolist()])
    
borda_df = pd.Series(borda_results)

In [313]:
bdf = borda_df.to_frame()
bdf = bdf.reset_index()
bdf.columns=["ID","TopConfigs"]

In [280]:
import numpy as np

def process_file(series):   
    
    values = series.values
    values = values[~np.isnan(values)]
    best_order = np.array(values).argsort()
    best_order = best_order[best_order != -1]
    stringified = ";".join([str(x) for x in best_order.tolist()])
    return stringified

processed = df.apply(process_file)
processed1 = dataframes[0].apply(process_file, axis=1)
processed2 = dataframes[1].apply(process_file, axis=1)

comparison = pd.DataFrame({
    "ID": processed.index,
    "processed": processed.values,
})


In [282]:
bdf

layout:xla:random:05ae41e26dd3c4c06390371a0423233c     17;106;494;398;400;498;259;274;358;341;143;267...
layout:xla:random:3e7156ac468dfb75cf5c9615e1e5887d     904;314;113;29;690;619;184;275;816;192;234;287...
layout:xla:random:5335ed13823b0a518ee3c79ba4425f34     545;81;916;522;214;730;178;779;179;583;445;416...
layout:xla:random:937ee0eb0d5d6151b7b8252933b5c1c9     324;920;604;13;485;621;93;702;641;417;935;18;9...
layout:xla:random:cd708819d3f5103afd6460b15e74eaf3     147;76;552;335;341;764;747;940;516;871;904;857...
layout:xla:random:db59a991b7c607634f13570d52ce885f     123;745;767;927;586;987;976;549;446;543;669;68...
layout:xla:random:e8a3a1401b5e79f66d7037e424f3b6df     425;699;571;72;392;808;817;227;433;676;304;127...
layout:xla:random:fbaa8bb6a1aed9988281085c91065c05     333;319;753;739;64;708;68;202;262;780;328;767;...
layout:xla:default:05ae41e26dd3c4c06390371a0423233c    472;989;726;532;1;106;817;606;17;712;498;440;3...
layout:xla:default:3e7156ac468dfb75cf5c9615e1e5887d    

In [283]:
processed_df = pd.DataFrame(
    {"ID": processed.index, "TopConfigs": processed.values}
)

In [315]:
BASE_FILE = "submissions/bqkc1jsp_xla_6aeax7xs_nlp.csv"
base_data = pd.read_csv(BASE_FILE)

In [316]:
joined = base_data.merge(bdf, on="ID", how="left")

joined["TopConfigs"] = joined["TopConfigs_y"].fillna(joined["TopConfigs_x"])
joined = joined.drop(columns=["TopConfigs_x", "TopConfigs_y"])

In [317]:
output = "submissions/xlabordacomp.csv"
joined.to_csv(output, index=False)

In [289]:
bdf.tail()

,ID,TopConfigs
29,layout:nlp:default:71b79ca6db513e7979c3702c595...,47;265;748;860;811;404;594;77;154;847;166;208;...
30,layout:nlp:default:7f6284ebe027b1e9a3850fc7038...,513;688;784;311;116;242;645;709;424;110;895;49...
31,layout:nlp:default:b2fdde3b7298090757864877410...,47;773;866;746;184;311;158;871;58;953;18;231;5...
32,layout:nlp:default:d15316c12eefdef1ba549eb4337...,900;835;35;52;567;178;696;63;657;199;460;193;5...
33,layout:nlp:default:f6c146fc5cf10be4f3accbaca98...,896;611;865;433;398;57;621;881;849;376;954;766...
